Car Pipeline:
Car Detection (YOLO) -> detect and classify driver and seatbelt (Custom) Not Done Yet -> license plate detection(YOLO) -> OCR (pytesseract)


In [ ]:
import sys
import os 
if 'google.colab' in sys.modules:
    !pip -q install ultralytics
    !pip -q install pytesseract
    !apt-get -qq install tesseract-ocr
    if not os.path.exists('license-plate-finetune-v1s.pt'):
        !wget https://huggingface.co/morsetechlab/yolov11-license-plate-detection/resolve/main/license-plate-finetune-v1s.pt


In [ ]:
from ultralytics import YOLO
from ultralytics.engine.results import Results
from pathlib import Path
import pytesseract
from PIL import Image
import numpy as np
from typing import Any

license_plate_detection_model = YOLO('license-plate-finetune-v1s.pt')

car_detection_model = YOLO("yolo11n.pt")


In [ ]:
def detect_cars(image_path: Path | str) -> list[np.ndarray]:
    # First, detect cars in the image
    car_results = car_detection_model.predict(source=image_path, classes=[2,5,7])  # COCO classes for car, bus, truck
    output = []
    for car in car_results[0].boxes:
        # Extract car bounding box
        x1, y1, x2, y2 = map(int, car.xyxy[0])
        car_image = car_results[0].orig_img[y1:y2, x1:x2]
        output.append(car_image)  
    return output

In [ ]:
def plate_detection(car_images: list[np.ndarray]) -> list[Results]:
    plate_results = license_plate_detection_model.predict(source=car_images)
    return plate_results

In [ ]:
def run_ocr_on_plate(plate_results: list[Results])->list[str]:
    ocr_results = []
    for plate in plate_results[0].boxes:
        x1, y1, x2, y2 = map(int, plate.xyxy[0])
        plate_image = plate_results[0].orig_img[y1:y2, x1:x2]
        
        # Convert numpy array to PIL Image for pytesseract
        pil_image = Image.fromarray(plate_image)
        ocr_result = pytesseract.image_to_string(pil_image, config='--psm 7')
        ocr_results.append(ocr_result.strip())
    return ocr_results


In [ ]:
def run_pipeline(image_path: Path | str):
    car_images = detect_cars(image_path)
    plate_results = plate_detection(car_images)
    return plate_results

In [ ]:
if 'google.colab' in sys.modules:
    if "bmwalleng.png" not in os.listdir("."):
      !wget https://github.com/AFAskar/dlimageclass/blob/main/bmwalleng.png?raw=true
      !mv bmwalleng.png?raw=true bmwalleng.png


In [ ]:

import json

images = ['./bmwalleng.png']
for img_path in images:
    plate_results = run_pipeline(img_path)
    ocr_results = run_ocr_on_plate(plate_results)    

plate_results[0].show()

with open('ocr_results.json', 'w') as f:
    json.dump(ocr_results, f)